In [6]:
import os
import pickle
from ase.db import connect
from ase.visualize import view

from GAMERNet.rnet.networks.reaction_network import ReactionNetwork
from GAMERNet.rnet.networks.surface import Surface

with open('../scripts/C2_Pt111/rxn_net.pkl', 'rb') as pickle_file:
    content = pickle.load(pickle_file)  # dict of elementary reactions

rxn_net = ReactionNetwork().from_dict(content)
print(rxn_net)
graph = rxn_net.graph
print(rxn_net.intermediates)

ReactionNetwork(250 intermediates, 44 closed-shell molecules, 1081 reactions)
Surface: Pt48(111)
Network Carbon cutoff: C2

{'264101': 264101(C2H6O4), '254101': 254101(C2H5O4), '254102': 254102(C2H5O4), '254103': 254103(C2H5O4), '244101': 244101(C2H4O4), '244102': 244102(C2H4O4), '244103': 244103(C2H4O4), '244104': 244104(C2H4O4), '244105': 244105(C2H4O4), '234101': 234101(C2H3O4), '234102': 234102(C2H3O4), '234103': 234103(C2H3O4), '234104': 234104(C2H3O4), '234105': 234105(C2H3O4), '234106': 234106(C2H3O4), '224101': 224101(C2H2O4), '224102': 224102(C2H2O4), '224103': 224103(C2H2O4), '224104': 224104(C2H2O4), '224105': 224105(C2H2O4), '214101': 214101(C2HO4), '214102': 214102(C2HO4), '214103': 214103(C2HO4), '204101': 204101(C2O4), '000000': 000000(Pt48), '010101': 010101(H), '264201': 264201(C2H6O4), '254201': 254201(C2H5O4), '254202': 254202(C2H5O4), '244201': 244201(C2H4O4), '244202': 244202(C2H4O4), '244203': 244203(C2H4O4), '244204': 244204(C2H4O4), '244205': 244205(C2H4O4), '23

In [9]:
closed_shell_atoms = []
for inter in rxn_net.intermediates.values():
    if inter.closed_shell == True:
        closed_shell_atoms.append(inter.molecule)
print(len(closed_shell_atoms))

44


In [10]:
view(closed_shell_atoms)

<Popen: returncode: None args: ['/home/smorandi/miniconda3/envs/gnn/bin/pyth...>

In [2]:
rxn_net.write_dotgraph(".", 'testC2.png')

KeyboardInterrupt: 

Look for intermediates with specified composition

In [7]:
rxn_net.search_inter_by_elements({'C':1, 'H':2, 'O':2})

(122101(CH2O2), 122102(CH2O2), 122103(CH2O2))

Look for all elementary steps involving a specific intermediate

In [8]:
rxn_net.search_ts(["100101"])

(Pt48+CH<->H+C,
 C2H3O2+Pt48<->CH3O2+C,
 C2H2O2+Pt48<->CH2O2+C,
 C2H2O2+Pt48<->CH2O2+C,
 C2HO2+Pt48<->CHO2+C,
 C2HO2+Pt48<->CHO2+C,
 C2O2+Pt48<->C+CO2,
 Pt48+C2H3O3<->CH3O3+C,
 C2H2O3+Pt48<->CH2O3+C,
 Pt48+C2HO3<->CHO3+C,
 C2O3+Pt48<->CO3+C,
 C2H3+Pt48<->CH3+C,
 C2H2+Pt48<->C+CH2,
 C2H+Pt48<->CH+C,
 C2+Pt48<->C,
 C2H3O+Pt48<->CH3O+C,
 Pt48+C2H2O<->CH2O+C,
 Pt48+C2H2O<->CH2O+C,
 C2HO+Pt48<->CHO+C,
 C2HO+Pt48<->CHO+C,
 C2O+Pt48<->CO+C,
 CHO+Pt48<->HO+C,
 CO+Pt48<->O+C)

# closed shell

In [29]:
def is_closed_shell_santi(self):
        """
        Check if a molecule CxHyOz is closed-shell or not.
        """
        graph = self.graph
        # print(graph.nodes()) list of node indexes, element symbol stored as "elem"
        molecule = self.molecule
        valence_electrons = {'C': 4, 'H': 1, 'O': 2}
        graph = graph.to_undirected()
        mol_composition = molecule.get_chemical_symbols()
        mol = {'C': mol_composition.count('C'), 'H': mol_composition.count('H'), 'O': mol_composition.count('O')} # CxHyOz

        if mol['C'] != 0 and mol['H'] == 0 and mol['O'] == 0: # Cx
                return False
        elif mol['C'] == 0 and mol['H'] != 0 and mol['O'] == 0: # Hy
                return True if mol['H'] == 2 else False
        elif mol['C'] == 0 and mol['H'] == 0 and mol['O'] != 0: # Oz
                return True if mol['O'] == 2 else False
        elif mol['C'] != 0 and mol['H'] == 0 and mol['O'] != 0: # CxOz
                return True if mol['C'] == 1 and mol['O'] in (1,2) else False
        elif mol['C'] != 0 and mol['H'] != 0: # CxHyOz (z can be zero)
            node_val = lambda graph: {node: (graph.degree(node), 
                                        valence_electrons.get(graph.nodes[node]["elem"], 0)) for node in graph.nodes()}
            num_unsaturated_nodes = lambda dict: len([node for node in dict.keys() if dict[node][0] < dict[node][1]])
            node_valence_dict = node_val(graph)
            if num_unsaturated_nodes(node_valence_dict): # all atoms are saturated
                return True
            elif num_unsaturated_nodes(node_valence_dict) == 1: # only one unsaturated atom
                return False
            else:
                saturation_condition = lambda dict: all(dict[node][0] == dict[node][1] for node in dict.keys())
                while saturation_condition(node_valence_dict) == False:
                    unsat_nodes = [node for node in node_valence_dict.keys() if node_valence_dict[node][0] < node_valence_dict[node][1]]
                    O_unsat_nodes = [node for node in unsat_nodes if graph.nodes[node]["elem"] == 'O']  # all oxygens unsaturated
                    if len(O_unsat_nodes) != 0: # only one unsaturated oxygen
                        for oxygen in O_unsat_nodes:
                            node_valence_dict[oxygen][0] += 1
                            # increase the valence of the oxygen neighbour by 1
                            for neighbour in graph.neighbors(oxygen): # only one neighbour
                                if node_valence_dict[neighbour][0] < node_valence_dict[neighbour][1]:
                                    node_valence_dict[neighbour][1] += 1
                                else:
                                    return False # O neighbour is saturated already
                    else: # CxHy
                         # select node with the highest degree
                        max_degree = max([node_valence_dict[node][0] for node in unsat_nodes])
                        max_degree_node = [node for node in unsat_nodes if node_valence_dict[node][0] == max_degree][0]
                        max_degree_node_unsat_neighbours = [neighbour for neighbour in graph.neighbors(max_degree_node) if neighbour in unsat_nodes]
                        if len(max_degree_node_unsat_neighbours) == 0: # all neighbours are saturated
                            return False
                        node_valence_dict[max_degree_node][0] += 1
                        node_valence_dict[max_degree_node_unsat_neighbours][0] += 1
                         
                            

                
        
        # # Getting the unsaturated nodes (if there are not unsaturated nodes, the molecule is closed-shell)
        # unsat_nodes = [node for node in graph.nodes() if graph.degree(node) < valence_electrons.get(graph.nodes[node]["elem"], 0)]

        # # If the graph only has Carbon as an element and not H or O, then it is open-shell
        # if not 'H' and 'O' in molecule.get_chemical_formula():
        #     print(f'System {molecule.get_chemical_formula()} is open-shell: only C atoms')
        #     return False 
        
        # # Specific case for O2
        # if not 'C' and 'H' in molecule.get_chemical_formula() and len(unsat_nodes) == 2:
        #     print(f'System {molecule.get_chemical_formula()} is closed-shell: Oxygen')
        #     return True 
        
        # # CO and CO2
        # if not 'H' in molecule.get_chemical_formula() and len(molecule.get_chemical_symbols()['C']) == 1 and len(molecule.get_chemical_symbols()['O']) in (1,2):
        #     print(f'System {molecule.get_chemical_formula()} is closed-shell: CO or CO2')
        #     return True
        
        # if unsat_nodes:
        #     # If the molecule has only one unsaturated node, then it is open-shell
        #     if len(unsat_nodes) == 1:
        #         print(f'System {molecule.get_chemical_formula()} is open-shell')
        #         return False 
        #     else:
        #         # Checking if there is one unsaturated node that does not have as neighbour another unsaturated node
        #         for node in unsat_nodes:
        #             # If the molecule has only one unsaturated node, then it is open-shell
        #             if not [n for n in graph.neighbors(node) if n in unsat_nodes]:
        #                 print(f'System {molecule.get_chemical_formula()} is open-shell: one node is unsaturated but does not have as neighbour another unsaturated node')
        #                 return False 
        #             else:
        #                 # Case for molecules where an unsaturated node is oxygen
        #                 if graph.nodes[node]["elem"] == 'O':
        #                     # Adding one bond order (valence electrons) to the oxygen node by adding it to the unsat_nodes list

In [30]:
is_closed_shell_santi(rxn_net.intermediates['121101'])

True